# Kinematic Wave Parameter Estimation

## Requirements

This module uses information from the USGS stations over the CONUS to generate a statistical model for estimating the kinematic wave alpha and beta parameters for a given grid. The already trained model is provided here.

This model requires basin averaged estimates of the following variables:
* Mean annual temperature (degrees C)
* Mean annual precipitation (in)
* Drainage area (km²) - basin.area.tif file from the average basin step


And the pixel by pixel variable:
* Relief ratio

### Comments

* Depending on your Sklearn version, it may be necessary to re-run the model training to maintain compatibility (EF5-KW-Modeling-master script).
* The raster nodata value is assumed to be -9999

In [3]:
from ef5_kw_estimate import ReadGrid, WriteGrid
import numpy as np
import pickle
from sklearn.impute import SimpleImputer
import rasterio


## Read in the input grids

In [4]:
# keepInfo=True keeps the projection and spatial reference information for this grid...
# to use when writing the output grids
# We need log basin area, so compute that here
basinArea = ReadGrid("input_grids/basin.area.tif",keepInfo=True)

basinArea = np.log10(basinArea)

temp     = ReadGrid("input_grids/mean_temp.avg.tif")
precip   = ReadGrid("input_grids/mean_precip.avg.tif")
precip = precip/12
rr       = ReadGrid("input_grids/relief.ratio.tif")


ERROR 1: PROJ: proj_create_from_database: datum not found
ERROR 1: PROJ: proj_create_from_database: datum not found
/var/folders/1z/2zs73sj958b3gtzwq06vr0280000gr/T/ipykernel_4064/3427228424.py:6: RuntimeWarning: invalid value encountered in log10
  basinArea = np.log10(basinArea)


In [5]:
print("Checking raster data...")
print("Max values:", np.nanmax([basinArea, temp, precip, rr], axis=1))
print("Min values:", np.nanmin([basinArea, temp, precip, rr], axis=1))

Checking raster data...
Max values: [  5.610286    29.254793   164.58334      0.19926934]
Min values: [-8.584308e-02 -9.999000e+03 -9.999000e+03 -9.999000e+03]


## Load the pickled models so we can run them on our data

In [6]:
scaler = pickle.load(open("scaler.p", "rb"), encoding='latin1')
alphamod = pickle.load(open("alpha_model.p", "rb"), encoding='latin1')
betamod = pickle.load(open("beta_model.p", "rb"), encoding='latin1')


/Users/vrobledodelgado/anaconda3/envs/vrobledod/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RobustScaler from version 1.5.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/vrobledodelgado/anaconda3/envs/vrobledod/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator SVR from version 1.5.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


## Run the models to estimate alpha & beta

In [7]:
print("Transforming to scaled parameter space")
pred_real = scaler.transform(np.column_stack((basinArea, temp, precip, rr)))


Transforming to scaled parameter space


In [8]:
print("Computing alpha")
alpha = alphamod.predict(pred_real)
alpha = np.power(10.0, alpha)
alpha.shape

Computing alpha


(1110510,)

In [9]:
print("Computing beta")
beta = betamod.predict(pred_real)


Computing beta


In [10]:
# Since the model extrapolates, we do a trick here to bound the minimum beta to 0.
bad = np.where(beta < 0.0)
beta[bad] = np.exp(beta[bad] * 10.0)

with rasterio.open("input_grids/FAC_ghana1km.tif") as src: #Change this to the appropriate FAC grid for your region
    fam = src.read(1)
# FAC grid is used to mask out areas that are not suitable for the model
mask = fam > 7 #is the minimum value in the FAC grid for this resolution, this parameter can change for different pixel resolutions
mask=np.reshape(mask,(1, -1))[:][0]
alpha[np.where(alpha>1000)]=np.nan
alpha[~mask] = np.nan
beta[~mask] = np.nan

## Write the new parameter grids to disk

In [12]:
import osgeo.osr as osr
import osgeo.gdal as gdal
from osgeo.gdalconst import *
##if you did not use keepInfo in the first block, you have to use this.
def GetGridInfo(gridIn):
    dem = gdal.Open(gridIn, GA_ReadOnly)
    if dem is None:
        raise ValueError(f"Error opening file: {gridIn}")

    gt = dem.GetGeoTransform()
    proj = dem.GetProjection()
    nx = dem.RasterXSize
    ny = dem.RasterYSize
    
    print(f"Extracted Grid Info from {gridIn}: nx={nx}, ny={ny}")
    return gt, proj, nx, ny

gt, proj, nx, ny = GetGridInfo("input_grids/basin.area.tif")

def WriteGrid2(gridOutName, dataOut, gt, proj, nx, ny):
    driver = gdal.GetDriverByName('GTiff')
    dst_ds = driver.Create(gridOutName, nx, ny, 1, gdal.GDT_Float32)
    
    if dst_ds is None:
        raise RuntimeError("Failed to create output file. Check the parameters.")

    dst_ds.SetGeoTransform(gt)
    dst_ds.SetProjection(proj)
    dst_ds.GetRasterBand(1).WriteArray(dataOut.reshape(ny, nx))
    dst_ds.GetRasterBand(1).SetNoDataValue(-9999.0)
    dst_ds = None
    print(f"Successfully wrote {gridOutName}")

# Call WriteGrid using metadata from `basinArea.tif`
WriteGrid2("./outputs/kw_alpha.tif", alpha, gt, proj, nx, ny)
WriteGrid2("./outputs/kw_beta.tif", beta, gt, proj, nx, ny)

Extracted Grid Info from input_grids/basin.area.tif: nx=914, ny=1215
Successfully wrote ./outputs/kw_alpha.tif
Successfully wrote ./outputs/kw_beta.tif


ERROR 1: PROJ: proj_create_from_database: datum not found
ERROR 1: PROJ: proj_create_from_database: datum not found


## Alpha0

For alpha0 estimation, the inputs are:
* Manning roughness coefficient
* Slope (in m/m units), and flow accumulation.

The slope is calculated using the DEM as an input, assuming geographic coordinates.

In [14]:
import rasterio
import numpy as np

# Define a function to convert degrees to meters
def degrees_to_meters(degrees_x, degrees_y, latitude):
    meters_per_degree_longitude = 111320 * np.cos(np.radians(latitude))
    meters_per_degree_latitude = 110574
    meters_x = degrees_x * meters_per_degree_longitude
    meters_y = degrees_y * meters_per_degree_latitude
    return meters_x, meters_y

# Read the DEM
with rasterio.open("input_grids/DEM_ghana1km.tif") as src:
    dem = src.read(1)
    transform = src.transform
    profile = src.profile

# Compute the pixel size in meters
pixel_size_x = transform[0]
pixel_size_y = -transform[4]

# Compute the gradient in meters
height, width = dem.shape
lat_center = transform[5] + (height // 2) * transform[4]

res_x_m, res_y_m = degrees_to_meters(pixel_size_x, pixel_size_y, lat_center)

# Compute the gradient
grad_y, grad_x = np.gradient(dem, res_y_m, res_x_m)

# Compute the slope
slope = np.sqrt(grad_x**2 + grad_y**2)

# Write the slope to a new raster
out_meta = profile.copy()
out_meta.update({"dtype": "float32", "nodata": None})

with rasterio.open("outputs/slope_m_m.tif", "w", **out_meta) as dest:
    dest.write(slope.astype("float32"), 1)

In [ ]:
import numpy as np
import rasterio
from rasterio.transform import Affine
from rasterio.crs import CRS

# Read the manning coefficient change this name to the appropriate manning coefficient grid for your region
with rasterio.open("input_grids/manning_n_1km.tif") as src:
    manning = src.read(1)
    transform = src.transform
    crs = src.crs

# Read the slope
with rasterio.open("outputs/slope_m_m.tif") as src:
    slope_grid = src.read(1)

# Read the flow accumulation
with rasterio.open("input_grids/FAC_ghana1km.tif") as src:
    facc = src.read(1)


# Compute the Chezy coefficient
COEM = 1.0 / manning
print(manning.shape)

# Compute the alpha0
alpha0 = np.zeros_like(facc, dtype=np.float32)
valid_mask = facc >= 0
print(facc.shape)
alpha0[valid_mask] = (1.0 / (COEM[valid_mask] * np.sqrt(slope_grid[valid_mask]))) ** (3 / 5)

# Clip the alpha0 values to a maximum of 1000
max_valid = alpha0[alpha0 < 1000].max()
alpha0[alpha0 >= 1000] = max_valid

# Write the alpha0 to a new raster
alpha0_grid = np.full_like(facc, np.nan, dtype=np.float32)
alpha0_grid[valid_mask] = alpha0[valid_mask]

output_meta = {
    'driver': 'GTiff',
    'height': alpha0_grid.shape[0],
    'width': alpha0_grid.shape[1],
    'count': 1,
    'dtype': 'float32',
    'crs': crs,
    'transform': transform
}

with rasterio.open('outputs/kw_alpha0.tif', 'w', **output_meta) as dst:
    dst.write(alpha0_grid, 1)

(1215, 914)
(1215, 914)


/var/folders/1z/2zs73sj958b3gtzwq06vr0280000gr/T/ipykernel_4064/2559801645.py:29: RuntimeWarning: divide by zero encountered in true_divide
  alpha0[valid_mask] = (1.0 / (COEM[valid_mask] * np.sqrt(slope_grid[valid_mask]))) ** (3 / 5)
